# Exploratory Data Analysis
#### Author : Navesh Kumar
#### Date : 21 Oct 2024

In [1]:
# specifying the location of the files
source_path = 'D:/vpi/intraday_orderbook_snapshots/intraday_orderbook_snapshots'

In [9]:
# Using below Libraries for analysis and visualisation
import pandas as pd

In [9]:
# read a file to understand the structure of the file
sample_file_path = f'{source_path}/snapshot_2022_3_20_7_40_0.parquet'
sample = pd.read_parquet(sample_file_path)

In [4]:
print(sample.info())
print(sample.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   DeliveryStart  607 non-null    datetime64[ns, UTC]
 1   Product        607 non-null    object             
 2   BQty           607 non-null    float64            
 3   Bid            607 non-null    object             
 4   Ask            607 non-null    object             
 5   AQty           607 non-null    float64            
 6   Level          607 non-null    int64              
dtypes: datetime64[ns, UTC](1), float64(2), int64(1), object(3)
memory usage: 33.3+ KB
None
              DeliveryStart                Product  BQty     Bid    Ask  AQty  \
0 2023-01-01 00:15:00+00:00  Intraday_Quarter_Hour   4.3   -7.98   1.86  24.2   
1 2023-01-01 00:15:00+00:00  Intraday_Quarter_Hour   8.8  -10.90   5.00  24.3   
2 2023-01-01 00:15:00+00:00  Intraday_Quarter_Hour   0.8 

#### Data Structure and Domain understanding
The file is a snapshot at a timestamp as noted by the file name </n>
</n>
Columns
</n>
1. DeliveryStart: Identifies the delivery time of the product </n>
2. Product : given Xbid market in the EU; these are the HH, QH and H blocks identifiers</n>
3. BQty : Bid volume</n>
4. Bid</n>
5. AQty: Offer Volume</n>
6. Ask: Offer</n>
7. Level : repeating 1 to 5: this represents the heirarchy of a data point in the market depth</n>

# Finding 1 : Descriptive analysis of data
The given data set contains market depth Only till 5 levels of Market Depth per instrument (DeliveryStart) per Product

# Finding 2 : Data quality and naming convention
### 2.1 Data Quality
The Products mentioned in this sample file indicate that the products being traded are
1. XBID Hourly, Half Hourly, Quater Hourly products delivering in thier respective times
2. Intraday Hourly, Half Hourly, Quater Hourly products delivering in thier respective times

<i>While</i> the file names indicate in its format of YEAR_MONTH_DAY_HOUR_MINUTE_SECOND that the snapshot of trades should belong to the year 2022, however the data within the files are from 2023 in the YYYY signature in DeliveryStart column</br>

Unless this data is specifically from futures market that may have a provision of trading such instruments (to best of my knowledge it does not exist) there is a <b><i>MISMATCH</i></b> in the file's name YYYY and the data it has captured in DeliveryStart. What makes my argument stronger is that the instruments have sufficient depth (5 levels of it) hence proving the mismatch as the products mentioned above are <b><i>SHORT TERM INSTRUMENT</i></b> that should be traded only near the <b><i>GATE CLOSURE</i></b>

### 2.2 Time Zone
I am doing my analysis w.r.t UTC. so for me Half Hour 1 will be 0 hour +00:30 while in the EU, where this exchange is presumably, it is +01:00. The file naming convention is based in UTC +01:00. Hence if I am analysing Half Hour 20; I am working with assumption of Delivery at 09:30:00 +00:00. the DeliveryStart column is based on this logic and follows the Gate Closure file-wise accordingly. However, the file naming convetion is in UTC, so the file that contains the last trade before 9.30 AM is in a file named _8_30. I will handle this TZ shift in the backend.

### Plan for Analysing the data
Since the data is available in timestamps, the continuity of the ticker is lost. We also have to account for the YYYY mismatch. Hence any analysis outside fundamental analsys should consider this discrepancy. My scope for this EDA is just fundamentals.</br>
To understand the nature of the trading and really gain insight the data needs to be made available at a product level, for a unique product
##### Important to note that this analysis focuses only on fundamentals
Hence it is important that the observation period should be restricted to the most liquid period of a trade which is near its gate closure </n>

<b>The Plan</b>
1. specify the dates to observe: Origin and End </n>
2. specify the Product Observation Start and Product Observation Start w.r.t. Gate Closure </n>
3. Get all the Snapshot files within the origin and end dates merged as one dataframe </n>
4. Filter product </n>

This will get us an answer to the question : <i> During Origin to End dates, what was the fundamental Behaviour of Product XYZ


# Collecting the data for analysis

## My Naming Convention
Products deliver at a fixed time interval. For Half hourly products, there are 48 deliveries throughout the day.

Since we want to keep the Date of Delivery and the delivery time itself flexible to observe the same delivery periods throughout the year we declare the instrument of the the product through its delivery date by simply indicating which one, chronologically, we are talking about. </n>

Example: Half Hour product that delivers at 0100 can simply be called 3 as its the 3rd half hour product delivering </br></n>

### YYYY file to data correction
While as a user I would want to analyse the data from a fundamental perspective, the error correction for file naming convention both in the EDA and in the Algo Framework will be handled in the background. This makes it easier for me to think of trading strategies relevant to 2023 (albiet simple) as the 2022 and 2023 markets are contrasting.
### File name TZ correction
The file name local storage search will offset time by one hour to account for this issue.
### Clock change
Files dont seem to have clock change issue.


In [2]:
# I want to analyse March 20th and 21st
# collecting the Period of the Year we want to analyse
Origin = [2023,5,2,0,0,0]
End =    [2023,5,3,23,0,0]
# On March 20th and 21st I am intrested in analysing the market behavour of Half Hour Instruments delivering 10 and 12 in the daytime
#declaring the products as a dict - same feature used in algo framework
product_dict = {
    'QH':[40,48]
}
# Declaring a Period of Liquidity w.r.t gate closure
# these are applicable to all selected products
POS = 720 # Product Observation Start time 6 hours before gate closure
POE = 0 # Product Observation End time 5 mins before gate closure
# these are now needed to narrow down the files that contains our data

In [3]:
# which can then be broken into 
product_category =[*product_dict.keys()][0]
_key_var = [*product_dict.keys()][0]
products = product_dict[_key_var]

In [4]:
from datetime import datetime, timedelta
# We have a description of the instrument and the product
# Now it is converted into DeliveryStart for data loading

# the same half hour will occur on every day between Origin and End
# hence we need to calculate all the delivery start dates for given products
# this is the first step towards constructing the product wise data
# This is top down approach for keeping the code flexible
# first we declare what we want as data
# then we construct the date ranges we need for each data
# then we select the appropriate file
def product_to_time_mapping(product_catogory,product_delivery_time_string):
    if product_catogory == 'HH':
        hour, minute = map(int, product_delivery_time_string.split('_')[3:5])
        product_number = (hour * 2) + (1 if minute == 0 else 2)
        return f"{product_catogory}{product_number:02}" , product_number
    
    if product_catogory == 'QH':
        hour, minute = map(int, product_delivery_time_string.split('_')[3:5])
        product_number = (hour * 4) + (minute // 15) + 1
        return f"{product_catogory}{product_number:02}", product_number


def products_to_be_traded_generator(products,product_catogory,origin,end):
    if product_catogory == 'HH':
        collect_products_to_be_traded = []
        delivery_period = origin
        # rounding to nearest delivery period
        m = delivery_period.minute
        if m == 0:
            delivery_period.replace(second = 0,microsecond = 0)
        elif m <= 30:
            delivery_period.replace(minute = 30, second = 0,microsecond = 0)
        else:
            (delivery_period + timedelta(hours=1)).replace(minute = 0, second = 0,microsecond = 0)

        while delivery_period <= end:
            collect_products_to_be_traded.append(delivery_period)
            delivery_period = delivery_period + timedelta(minutes = 30)

    if product_catogory == 'QH':
        collect_products_to_be_traded = []
        delivery_period = origin
        m = delivery_period.minute
        if m % 15 == 0:
            delivery_period = delivery_period.replace(second=0, microsecond=0)
        else:
            # Calculate the nearest quarter-hour
            minute_adjustment = (15 - m % 15) % 15
            delivery_period = delivery_period + timedelta(minutes=minute_adjustment)
            delivery_period = delivery_period.replace(second=0, microsecond=0)

        # Collect quarter-hourly products
        while delivery_period <= end:
            collect_products_to_be_traded.append(delivery_period)
            delivery_period = delivery_period + timedelta(minutes=15)

    #convert the datetime into string for getting data
    str_products_to_be_traded = []
    for product in collect_products_to_be_traded:
        str_products_to_be_traded.append(f"{product.year}_{str(product.month)}_{str(product.day)}_{str(product.hour)}_{str(product.minute)}_{str(product.second)}")

    products_to_be_traded = []
    #find the product number for this delivery period
    for product_string in str_products_to_be_traded:
        product_map_id , check_entry_prd = product_to_time_mapping(product_catogory,product_string)
        #origin to end will contain all the delivery periods 
        # filtering the ones delcared in product
        if check_entry_prd in products:
            products_to_be_traded.append(f"{product_map_id}-{product_string}")
    
    return products_to_be_traded


products = products
product_category = product_category
origin = datetime(*Origin) 
end = datetime(*End) 
products_to_be_traded = products_to_be_traded_generator(products,product_category,origin,end)



We now have the full Schedule of the Products we need </br>
Using an element of the list we can get the data that we want to observe

In [8]:
#select a product for analysis
current_product = products_to_be_traded[1]

In [6]:
import os
import pandas as pd
from datetime import timezone
def GetData(current_product,POS,POE,product_category):
    prd_id , deliveryStart = current_product.split('-')

    #deliveryStart with POS and POE will locate the relevant files
    # prd_id will filter the relevant data from the files
    deliveryStart_dt = datetime(*[*map(lambda _d:int(_d),[*deliveryStart.split('_')])])

    # IMPORTANT!!: CORRECTION #1 : Reducing date by 1 year to find the file
    deliveryStart_dt = deliveryStart_dt - timedelta(days=365) # 2022 and 2023 are not leap years

    # IMPORTANT!!: CORRECTION #1 : Reducing date by 1 year to find the file
    deliveryStart_dt = deliveryStart_dt - timedelta(minutes=60) # TZ -01:00

    # Gate Closure Calculations is kept at zero as the snapshots are at 5 min interval and so is the GC
    deliveryStart_dt_GC = deliveryStart_dt + timedelta(minutes=0)

    #calculating the POS and POE
    POS_dt = deliveryStart_dt_GC - timedelta(minutes=POS)
    POE_dt = deliveryStart_dt_GC - timedelta(minutes=POE)

    # name of the files we need to collect
    collect_files = []
    collect_this = POS_dt
    while collect_this <= POE_dt:
        collect_this_str = f"snapshot_{collect_this.year}_{collect_this.month}_{collect_this.day}_{collect_this.hour}_{collect_this.minute}_{collect_this.second}.parquet"
        collect_files.append(collect_this_str)
        collect_this = collect_this + timedelta(minutes=5)

    # get the files from the data location    
    matching_files_dataframes = []
    for file in collect_files:
        file_path = os.path.join(source_path, file)
        df = pd.read_parquet(file_path)
        file_str = os.path.splitext(file)[0].replace('snapshot_', '')
        file_str = datetime(*[*map(lambda _d:int(_d) ,file_str.split('_')) ])
        df['timestamp'] = file_str
        matching_files_dataframes.append(df)

    if matching_files_dataframes:
        combined_df = pd.concat(matching_files_dataframes, ignore_index=True)
    else:
        combined_df = pd.DataFrame()

    all_products = {
                    'HH': 'XBID_Half_Hour',
                    'QH': 'XBID_Quarter_Hour',
                    'H' : 'XBID_Hour'
                    }
    filter_product = all_products[product_category]
    combined_df = combined_df[combined_df['Product'] == filter_product]

    # filtering the instrument
    # formatting filter_instrument for filter query
    
    filter_instrument = deliveryStart_dt
    # IMPORTANT!! : Re-Correction of datetime back to UTC +00:00
    filter_instrument = filter_instrument + timedelta(minutes=60)
    # IMPORTANT!! : Re-Correction of datetime back to 2023
    filter_instrument = filter_instrument + timedelta(days=365)
    filter_instrument = filter_instrument.replace(tzinfo=timezone.utc)
    filter_instrument = pd.Timestamp(filter_instrument)

    combined_df = combined_df[combined_df['DeliveryStart']==filter_instrument]

    # Important!! : Correcting the timestamp bringing it back to 2023
    combined_df['timestamp'] = combined_df['timestamp'] + pd.DateOffset(days=365)
    # Important!! : Correcting the timestamp for time zone
    combined_df['timestamp'] = combined_df['timestamp'] + pd.DateOffset(hours=1)

    return combined_df


In [9]:
foundation_data = GetData(current_product,POS,POE,product_category)

In [7]:
#split product and date
prd_id , deliveryStart = current_product.split('-')

#deliveryStart with POS and POE will locate the relevant files
# prd_id will filter the relevant data from the files
deliveryStart_dt = datetime(*[*map(lambda _d:int(_d),[*deliveryStart.split('_')])])

# IMPORTANT!!: CORRECTION #1 : Reducing date by 1 year to find the file
deliveryStart_dt = deliveryStart_dt - timedelta(days=365) # 2022 and 2023 are not leap years

# IMPORTANT!!: CORRECTION #1 : Reducing date by 1 year to find the file
deliveryStart_dt = deliveryStart_dt - timedelta(minutes=60) # TZ -01:00

# Gate Closure Calculations is kept at zero as the snapshots are at 5 min interval and so is the GC
deliveryStart_dt_GC = deliveryStart_dt + timedelta(minutes=0)

#calculating the POS and POE
POS_dt = deliveryStart_dt_GC - timedelta(minutes=POS)
POE_dt = deliveryStart_dt_GC - timedelta(minutes=POE)


In [8]:
#since snapshots are at 5 minutes interval then all the files between these two times should be ingressed
collect_files = []
collect_this = POS_dt
while collect_this <= POE_dt:
    collect_this_str = f"snapshot_{collect_this.year}_{collect_this.month}_{collect_this.day}_{collect_this.hour}_{collect_this.minute}_{collect_this.second}.parquet"
    collect_files.append(collect_this_str)
    collect_this = collect_this + timedelta(minutes=5)

In [9]:
# we collect the files from source
import os
import pandas as pd
matching_files_dataframes = []
for file in collect_files:
    file_path = os.path.join(source_path, file)
    df = pd.read_parquet(file_path)
    file_str = os.path.splitext(file)[0].replace('snapshot_', '')
    file_str = datetime(*[*map(lambda _d:int(_d) ,file_str.split('_')) ])
    df['timestamp'] = file_str
    matching_files_dataframes.append(df)

if matching_files_dataframes:
    combined_df = pd.concat(matching_files_dataframes, ignore_index=True)
else:
    combined_df = pd.DataFrame()

# Finding 1
XBID markets have more liquidity than Intraday Local markets </br>
For this analysis we will focus only on XBID markets </br>
<i>We will also develop the algoframework for just XBID</i> however the framework is flexible enough to be extended for local markets as well

In [10]:
# Multiple products and instruments are trading in this snapshot
# we need to filter the desired product and desired instrument
#
all_products = {
    'HH': 'XBID_Half_Hour',
    'QH': 'XBID_Quarter_Hour',
    'H' : 'XBID_Hour'
}

In [11]:
# filterting the product we are analysing
filter_product = all_products[product_category]

In [12]:
combined_df = combined_df[combined_df['Product'] == filter_product]

In [13]:
# filtering the instrument
# formatting filter_instrument for filter query
from datetime import timezone
filter_instrument = deliveryStart_dt
# IMPORTANT!! : Re-Correction of datetime back to UTC +00:00
filter_instrument = filter_instrument + timedelta(minutes=60)
# IMPORTANT!! : Re-Correction of datetime back to 2023
filter_instrument = filter_instrument + timedelta(days=365)
filter_instrument = filter_instrument.replace(tzinfo=timezone.utc)
filter_instrument = pd.Timestamp(filter_instrument)


In [15]:
combined_df = combined_df[combined_df['DeliveryStart']==filter_instrument]

In [16]:
# Important!! : Correcting the timestamp
combined_df['timestamp'] = combined_df['timestamp'] + pd.DateOffset(days=365)

In [17]:
combined_df['timestamp'] = combined_df['timestamp'] + pd.DateOffset(hours=1)